# Solar Energy Consumption and Production Analysis
This notebook analyzes solar energy consumption and production. It calculates various metrics, including PV production, unmet demand, and performance indicators, under different conditions (e.g., number of panels and batteries).


### Solar + Wind Energy Consumption and Production Analysis for Grenoble City (For one day)

In [ ]:
# --------------------------------
# Required Library Installation
# --------------------------------

import sys
import subprocess

# List of required Python libraries for the project
required_libraries = [
    'pandas',            # Data processing and manipulation
    'numpy',             # Numerical calculations
    'plotly',            # Interactive visualizations
    'ipywidgets',        # Interactive widgets for user input
    'matplotlib',        # Plotting capabilities 
]

# Function to check and install missing libraries
def install_missing_libraries(libraries):
    for lib in libraries:
        try:
            __import__(lib)
        except ImportError:
            print(f"Installing missing library: {lib}")
            subprocess.check_call([sys.executable, "-m", "pip", "install", lib])
        else:
            print(f"✔ {lib} is already installed.")

# Run the function to install missing libraries
install_missing_libraries(required_libraries)


In [1]:
# main.ipynb
# ============================
# Solar + Wind Energy Consumption and Production Analysis for Grenoble City
# ============================
#
# Author: Arbaz KHALID & Tshephang Komana
# Date: 2024-12-28
#
# Description:
# This notebook performs a comprehensive analysis of solar and wind energy
# consumption and production for the entire city of Grenoble for the year 2019.
# Users can adjust the number of PV panels, wind turbines, and batteries, as well as
# select specific dates to visualize metrics such as unmet demand, excess energy,
# and battery SOC evolution.
#
# --------------------------------
# 1. Environment Setup
# --------------------------------
import os
import sys
import logging

import pandas as pd
import numpy as np
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

# Import the custom analysis script
import solar_analysis_script as sas

# (Optional) Add custom CSS to prevent label truncation (like in the weekly code)
HTML('''
<style>
    .widget-label {
        min-width: 200px !important;
        white-space: nowrap !important;
        overflow: visible !important;
        text-overflow: unset !important;
    }
</style>
''')

# --------------------------------
# 2. Configuration and File Paths
# --------------------------------
# Paths to data files
CONSUMPTION_FILE = 'Grenoble_city_load_1.csv'
PV_FILE = 'One_PV_Panel_Production.csv'
WIND_FILE = 'wind_production_for_one_turbine.csv'  

# Date ranges for the year 2019
CONSUMPTION_START_DATE = '2019-01-01 00:00:00'
CONSUMPTION_END_DATE   = '2019-12-31 23:59:59'

# Initial values & parameters from the script
INITIAL_NUM_PANELS   = sas.INITIAL_NUM_PANELS
INITIAL_NUM_BATTERIES = 500       # Adjust as needed
INITIAL_NUM_TURBINES = sas.INITIAL_NUM_TURBINES  

# --------------------------------
# 3. Load and Merge Data
# --------------------------------
print("Loading and preprocessing consumption data...")
try:
    city_consumption = sas.load_city_consumption(
        consumption_file_path=CONSUMPTION_FILE,
        consumption_sep=';',
        consumption_date_col='Date',
        consumption_time_col='Heures',
        consumption_value_col='Consommation(MWh)'
    )
    print("Consumption Data (head):")
    print(city_consumption.head())
except Exception as err:
    print(f"Failed to load consumption data: {err}")
    raise

print("\nLoading and preprocessing PV production data...")
try:
    city_pv_production = sas.load_city_pv_production(
        pv_file_path=PV_FILE,
        consumption_start_date=CONSUMPTION_START_DATE,
        consumption_end_date=CONSUMPTION_END_DATE,
        sep=',',
        time_start_col='time',
        time_end_col='local_time',
        electricity_col='electricity'
    )
    print("PV Production Data (head):")
    print(city_pv_production.head())
except Exception as err:
    print(f"Failed to load PV production data: {err}")
    raise

print("\nLoading and preprocessing Wind production data...")
try:
    city_wind_production = sas.load_city_wind_production(
        wind_file_path=WIND_FILE,
        consumption_start_date=CONSUMPTION_START_DATE,
        consumption_end_date=CONSUMPTION_END_DATE,
        sep=',',
        time_start_col='time',
        time_end_col='local_time',
        electricity_col='wind_electricity'
    )
    print("Wind Production Data (head):")
    print(city_wind_production.head())
except Exception as err:
    print(f"Failed to load wind production data: {err}")
    raise

print("\nMerging consumption, PV, and wind production data...")
try:
    # Merge all three data sources into one DataFrame
    city_merged_data = sas.merge_city_data(
        consumption=city_consumption,
        pv_production=city_pv_production,
        wind_production=city_wind_production,
        num_panels=INITIAL_NUM_PANELS,
        num_turbines=INITIAL_NUM_TURBINES
    )
    print("Merged Data (head):")
    print(city_merged_data.head())
except Exception as err:
    print(f"Failed to merge data: {err}")
    raise

# --------------------------------
# 4. Create Interactive Widgets with Updated Slider Style
# --------------------------------
slider_style = {'description_width': '150px'}
slider_layout = widgets.Layout(
    width='100%',
    min_width='400px',
    margin='0 0 0 0px'
)

num_panels_slider = widgets.IntSlider(
    value=INITIAL_NUM_PANELS,
    min=0,
    max=1e7,
    step=1e3,
    description='Number of PV Panels:',
    continuous_update=False,
    style=slider_style,
    layout=slider_layout
)

num_turbines_slider = widgets.IntSlider(
    value=INITIAL_NUM_TURBINES,
    min=0,
    max=1000,
    step=10,
    description='Number of Wind Turbines:',
    continuous_update=False,
    style=slider_style,
    layout=slider_layout
)

num_batteries_slider = widgets.IntSlider(
    value=INITIAL_NUM_BATTERIES,
    min=0,
    max=1e4,
    step=100,
    description='Number of Battery Units:',
    continuous_update=False,
    style=slider_style,
    layout=slider_layout
)

month_dropdown = widgets.Dropdown(
    options=[(pd.to_datetime(str(month), format='%m').strftime('%B'), month)
             for month in range(1, 13)],
    value=1,
    description='Month:',
    style={'description_width': '100px'},
    layout=widgets.Layout(width='400px')
)

date_dropdown = widgets.Dropdown(
    options=[],
    value=None,
    description='Date:',
    style={'description_width': '140px'},
    layout=widgets.Layout(width='500px')
)

message_output = widgets.Output()
plot_output = widgets.Output()

controls_box = widgets.VBox([
    widgets.VBox([
        num_panels_slider,
        num_turbines_slider,
        num_batteries_slider
    ], layout=widgets.Layout(
        justify_content='space-around',
        width='100%',
        padding='1px 2px'
    )),
    widgets.HBox([month_dropdown, date_dropdown],
                 layout=widgets.Layout(
                     justify_content='space-around',
                     width='100%',
                     padding='1px 2px'
                 ))
], layout=widgets.Layout(
    width='50%',
    padding='10px 0',
    border='1px solid #e0e0e0',
    margin='0px 0'
))

display(controls_box, plot_output)

# --------------------------------
# 5. Update Date Dropdown Dynamically Based on Selected Month
# --------------------------------
def populate_date_dropdown(change):
    """
    Based on the selected month, populate the date dropdown
    with the available dates from the merged dataset.
    """
    selected_month = change['new']
    month_filtered = city_merged_data[city_merged_data.index.month == selected_month]
    dates_in_month = month_filtered.index.normalize().unique()
    date_options = [date.strftime('%Y-%m-%d') for date in dates_in_month]
    
    with message_output:
        message_output.clear_output()
        if date_options:
            date_dropdown.options = date_options
            date_dropdown.value = date_options[0]
            print(f"Available dates in {pd.to_datetime(str(selected_month), format='%m').strftime('%B')}:")
            print(date_options)
        else:
            date_dropdown.options = []
            date_dropdown.value = None
            print(f"No data available for {pd.to_datetime(str(selected_month), format='%m').strftime('%B')}.")

month_dropdown.observe(populate_date_dropdown, names='value')
populate_date_dropdown({'new': month_dropdown.value})

# --------------------------------
# 6. Plot Handler Function
# --------------------------------
def graph_handler(num_panels, num_turbines, num_batteries, selected_date):
    """
    Handler function to clear outputs, call update_graph, and display the results.
    """
    with plot_output:
        plot_output.clear_output()
        if not selected_date:
            with message_output:
                message_output.clear_output()
                print("[Error] No date selected. Please select a valid date.")
            return
        
        # Generate the figure and SOC message from the analysis script
        fig, soc_message = sas.update_graph(
            num_panels=num_panels,
            num_turbines=num_turbines,
            num_batteries=num_batteries,
            selected_date=selected_date,
            city_merged_data=city_merged_data,
            soc_tracker=sas.soc_tracker,
            initial_num_panels=sas.INITIAL_NUM_PANELS,
            initial_num_turbines=sas.INITIAL_NUM_TURBINES,
            initial_soc=sas.INITIAL_SOC,
            battery_capacity_per_unit=sas.BATTERY_CAPACITY_PER_UNIT,
            converter_efficiency=sas.CONVERTER_EFFICIENCY
        )
        
        # Display the Plotly figure (if available)
        if fig:
            fig.show()
        else:
            print("No figure to display.")
    
    # Display SOC message in the message output area
    with message_output:
        message_output.clear_output()
        if soc_message:
            print(soc_message)

# --------------------------------
# 7. Connect the Widgets to the Plotting Function
# --------------------------------
interactive_out = widgets.interactive_output(
    graph_handler,
    {
        'num_panels': num_panels_slider,
        'num_turbines': num_turbines_slider,
        'num_batteries': num_batteries_slider,
        'selected_date': date_dropdown
    }
)

display(interactive_out)


Loading and preprocessing consumption data...
Consumption Data (head):
Datetime
2019-01-01 00:00:00    580000.0
2019-01-01 00:15:00    572000.0
2019-01-01 00:30:00    563000.0
2019-01-01 00:45:00    563000.0
2019-01-01 01:00:00    558000.0
Name: Consumption_kWh, dtype: float64

Loading and preprocessing PV production data...
PV Production Data (head):
Time_Start
2019-01-01 00:00:00    0.0
2019-01-01 01:00:00    0.0
2019-01-01 02:00:00    0.0
2019-01-01 03:00:00    0.0
2019-01-01 04:00:00    0.0
Name: PV_Production_kWh, dtype: float64

Loading and preprocessing Wind production data...
Wind Production Data (head):
Time_Start
2019-01-01 00:00:00     923.138
2019-01-01 01:00:00     942.556
2019-01-01 02:00:00    1064.198
2019-01-01 03:00:00    1328.341
2019-01-01 04:00:00    1625.839
Name: Wind_Production_kWh, dtype: float64

Merging consumption, PV, and wind production data...
Merged Data (head):
                     Consumption_kWh  PV_Production_kWh  Wind_Production_kWh
2019-01-01 00:00

Output()

Output()